In [1]:
import torch
import torch.nn as nn

In [3]:
# possibly add other kinds of vgg?
class VGG16(nn.Module):
    def __init__(self, nout):
        super().__init__()
        self.conv_1a = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=(3,3), padding=1)
        self.conv_1b = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3,3), padding=1)
        self.conv_2a = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3,3), padding=1)
        self.conv_2b = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=(3,3), padding=1)
        self.conv_3a = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=(3,3), padding=1)
        self.conv_3b = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=(3,3), padding=1)
        self.conv_4a = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=(3,3), padding=1)
        self.conv_4b = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=(3,3), padding=1)
        self.relu = nn.ReLU(True)
        self.pool = nn.MaxPool2d(2, 2)
        self.linear_1 = nn.Linear(512, 512)
        self.linear_2 = nn.Linear(512, 512)
        self.linear_3 = nn.Linear(512, nout)
        self.dropout = nn.Dropout()
        self.batchnorm_64 = nn.BatchNorm2d(64)
        self.batchnorm_128 = nn.BatchNorm2d(128)
        self.batchnorm_256 = nn.BatchNorm2d(256)
        self.batchnorm_512 = nn.BatchNorm2d(512)
        
    def forward(self, x):
        x = self.relu(self.batchnorm_64(self.conv_1a(x)))
        x = self.relu(self.batchnorm_64(self.conv_1b(x)))
        x = self.pool(x)
        x = self.relu(self.batchnorm_128(self.conv_2a(x)))
        x = self.relu(self.batchnorm_128(self.conv_2b(x)))
        x = self.pool(x)
        x = self.relu(self.batchnorm_256(self.conv_3a(x)))
        x = self.relu(self.batchnorm_256(self.conv_3b(x)))
        x = self.relu(self.batchnorm_256(self.conv_3b(x)))
        x = self.pool(x)
        x = self.relu(self.batchnorm_512(self.conv_4a(x)))
        x = self.relu(self.batchnorm_512(self.conv_4b(x)))
        x = self.relu(self.batchnorm_512(self.conv_4b(x)))
        x = self.pool(x)
        x = self.relu(self.batchnorm_512(self.conv_4b(x)))
        x = self.relu(self.batchnorm_512(self.conv_4b(x)))
        x = self.relu(self.batchnorm_512(self.conv_4b(x)))
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.relu(self.linear_1(x))
        x = self.dropout(x)
        x = self.relu(self.linear_2(x))
        x = self.dropout(x)
        x = self.linear_3(x)
        return x